# Import

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import copy
import math
import json
from scripts import utils
import importlib
import matplotlib.pyplot as plt
MAIN_DIR = "C:/Users/nourisa/Downloads/testProjs/proteomics_MSC"
SIG_FILE = os.path.join(MAIN_DIR, 'statistical_analysis_2/DiffExp_tables/TopTable_TimeCourse_SVD.csv') # dir for the sig analysis
OUTPUT_DIR = os.path.join(MAIN_DIR,'results')
STATANALYSIS = os.path.join(OUTPUT_DIR,'statAnalysis')
sys.path.insert(0,MAIN_DIR)

#- which 
    

# Edit the original df to create primary df 
Read the original proteomics data, process it by:
renaming the missing symbols
changing column names (to make it easier to follow)
removing rows with all zeros

In [ ]:
%run scripts/edit_data/edit.py

# Post statistical analysis

In [1]:
%run scripts/post_statistical_analysis/process.py

43


# Enrichment analysis
Run this when entichment analysis (STRING) data is available

## convert differnt gene annotation to one another. protname (UniProt name) and genename

In [ ]:
# geneID (entrenz gene id). https://www.uniprot.org/id-mapping/ 
from scripts import annotaion_map
import json
importlib.reload(annotaion_map)
importlib.reload(utils)
with open(os.path.join(MAIN_DIR, 'results/postprocess','DE_protnames.txt')) as f:
    DE_protnames = eval(f.read())['DE_protnames']

map_protname_geneID = annotaion_map.protname_geneID(DE_protnames)
with open(os.path.join(MAIN_DIR,'results/postprocess','map_protname_geneID.json'),'w') as f:
    f.write(json.dumps({'map':map_protname_geneID}))
# map protname to genename according to string: because it is different using the website
string_map = pd.read_csv(os.path.join(MAIN_DIR,'results/enrichment_analysis/string_mapping.tsv'), sep='\t',index_col=False)
map_protname_genename = {key:value for key,value in zip(string_map['queryItem'], string_map['preferredName'])}
# check if string imported all protnames successfully
try:
    assert(len(map_protname_genename)==len(DE_protnames))
except:
    for prot in DE_protnames:
        if prot not in string_map['queryItem'].values:
            print(f'prot {prot} was not mapped in string')
with open(os.path.join(MAIN_DIR,'results/postprocess','map_protname_genename.json'),'w') as f:
    f.write(json.dumps({'map':map_protname_genename}))
# rest
map_genename_protname = {value:key for value, key in zip(map_protname_genename.values(),map_protname_genename.keys())}
with open(os.path.join(MAIN_DIR,'results/postprocess','map_genename_protname.json'),'w') as f:
    f.write(json.dumps({'map':map_genename_protname}))
map_genename_geneID = {key:map_protname_geneID[value] for key, value in map_genename_protname.items()}
with open(os.path.join(MAIN_DIR,'results/postprocess','map_genename_geneID.json'),'w') as f:
    f.write(json.dumps({'map':map_genename_geneID}))
# write geneIDs to a file
with open(os.path.join(MAIN_DIR, 'results/postprocess','DE_geneIDs.json'),'w') as f:
    f.write(json.dumps({'geneIDs':list(map_protname_geneID.values())}))
# write genenames to a file
DE_genenames = list(map_protname_genename.values())
DE_genenames_str = ""
for gene in DE_genenames:
    DE_genenames_str+=gene + ","
with open(os.path.join(MAIN_DIR,'results/postprocess','DE_genenames.json'),'w') as f:
    json.dump({"DE_genenames":DE_genenames},f)

## format strings enrichment data  for plotting 
to plot enrichment results in r

In [ ]:
def reireive_string_data_F(enrich_type):
    '''
        Reads string EA data such as enrichment.Function
    '''
    FILE =  os.path.join(MAIN_DIR, 'results/enrichment_analysis/enrichment.all.tsv')
    data = pd.read_csv(FILE,sep='\t',index_col=False)
    # keep the specified enrichment type
    filter_ = data.loc[:,'#category'] == enrich_type
    data = data.loc[filter_,:]
    data.drop('#category',axis=1, inplace=True)
    # remove some columns
    data.drop('matching proteins in your network (IDs)',axis=1,inplace=True) 
    data.drop('background gene count',axis=1,inplace=True)
    # replace string gene name with uniprotname
    data.columns = ['ID','Description', 'ProteinCount', 'Strength','FDR','genenames']
    unitprotnames = []
    for row in data['genenames']:
        a = ''
        for item in row.split(','):
            a+=map_genename_protname[item]+','
        unitprotnames.append(a[0:-1])
    data['genenames'] = unitprotnames
    data.loc[:,'ProteinRatio'] = data['ProteinCount']/len(DE_protnames)
    # not useful but required as input to the graph
    data['pvalue'] = data['FDR'] 
    data['qvalue'] = data['pvalue'] 
    data.rename(columns={'genenames':'geneID'}, inplace=True)
    cut_t = 40
    short_descript = []
    for item in data.loc[:,'Description']:
        if len(item)>cut_t:
            aa =  item[0:cut_t] + '...'
        else:
            aa = item
        short_descript.append(aa)
    return data
data_process = reireive_string_data_F('GO Process')
data_process.to_csv(os.path.join(OUTPUT_DIR,'enrichment_analysis','enrichment_process.csv'),index=False)
print('number of enriched process tersm ',len(data_process))

data_function = reireive_string_data_F('GO Function')
data_function.to_csv(os.path.join(OUTPUT_DIR,'enrichment_analysis','enrichment_function.csv'),index=False)
print('number of enriched function tersm ',len(data_function))

data_component = reireive_string_data_F('GO Component')
data_component.to_csv(os.path.join(OUTPUT_DIR, 'enrichment_analysis','enrichment_component.csv'),index=False)
print('number of enriched component tersm ',len(data_component))

data_uniprot = reireive_string_data_F('UniProt Keywords')
data_uniprot.to_csv(os.path.join(OUTPUT_DIR, 'enrichment_analysis','enrichment_keyword.csv'))
print('number of enriched UniProt keywords terms ',len(data_uniprot))

data_reactome = reireive_string_data_F('Reactome')
data_reactome.to_csv(os.path.join(OUTPUT_DIR, 'enrichment_analysis','enrichment_reactome.csv'))
print('number of enriched Reactome tersm ',len(data_reactome))

## plot enriched terms

In [ ]:
importlib.reload(utils)
#- assignments
size_tag = 'Strength'
color_tag = 'FDR'
xlabel = 'ProteinCount'
#- data and their names
datas_1 = [data_process]
tags_1 = ['GO: Process'] # also shape tag
marker_types_1= ['o']
datas_2 = [data_component]
tags_2 = ['GO: Component'] # also shape tag
marker_types_2= ['o']
datas_3 = [data_function, data_uniprot, data_reactome]
tags_3 = ['GO: Biological Function', 'UniProt Keyword', 'Reactome Pathways'] # also shape tag
marker_types_3= ['*', 's', 'p']

fig = utils.enrichplotplot(datas_1, tags_1, size_t, color_t, xlabel, marker_types_1, figsize=(6.2,5), legend_color=True,legend_size=True, legend_marker=False,title='GO: Biological Process')
fig.savefig(os.path.join(OUTPUT_DIR, 'enrichment_analysis', 'biological_process.png'), dpi=300, transparent=True)
fig = utils.enrichplotplot(datas_2, tags_2, size_t, color_t, xlabel, marker_types_2, figsize=(6,5),legend_color=True,legend_size=True, legend_marker=False, title='GO: Biological Component')
fig.savefig(os.path.join(OUTPUT_DIR, 'enrichment_analysis', 'biological_component.png'), dpi=300, transparent=True)
fig = utils.enrichplotplot(datas_3, tags_3, size_t, color_t, xlabel, marker_types_3, figsize=(6,3.5),legend_color=True,legend_size=True, legend_marker=True, title='')
fig.savefig(os.path.join(OUTPUT_DIR, 'enrichment_analysis', 'multiple.png'), dpi=300, transparent=True, bbox_inches='tight')

In [ ]:
data_process

In [ ]:
df_clusters = pd.read_csv(os.path.join(MAIN_DIR,'string','string_MCL_clusters.tsv'),sep='\t')
df_clusters

# Noncategorized 

In [ ]:
# plot the time series of sig prots
utils.plot_time_series(df, prots=sig_prots, c_tag='ctr_', s_tag='mg_', p_ID='Entry', time=specs['time'], ee=0.5)

In [ ]:
# visualize the sig proteins for original and imputed versions
from scripts import utils
candids = sig_prots[0:10]
utils.plot_time_series_mutual(df1=df,df2=df_imput, prots=candids, )

plot time serties of Moreneo's sig prots and check which ones are detected in our analysis

In [ ]:
from scripts import utils
importlib.reload(utils)
moreneo_sigs = ['SDF4','NUFIP2','BUB3','HMGA2','KIF5B','DYNC1I2','DCTN1','S100A10','SYNPO2','MARCKS','THY1','HUWE1','PRPF8','RPS18','GCN1L1','RPL14','RBM8A','RPL23A','CANX','DLST','PDHB','ALDOC','APOA1','GGH','UAP1','ENO2','BAG3','HNRNPAB','BASP1','THRAP3','LRPPRC','CSE1L','PABPC4']
utils.plot_time_series(df, prots=moreneo_sigs[0:len(moreneo_sigs)], c_tag='ctr_', s_tag='mg_', p_ID='Gene', time=specs['time'], ee=0.5)

In [ ]:
# check the overlap between Moreneo's sig prots and this study
df_overlap = df_sig.loc[df_sig['Gene'].isin(moreneo_sigs)]
prots_overlap = df_overlap['Entry'].values
print(prots_overlap)
# utils.plot_time_series(df, prots=prots_overlap, c_tag='ctr_', s_tag='mg_', p_ID='Gene', time=specs['time'], ee=0.5)
# print(prots_overlap)
df_sig.head()

In [ ]:
# imputing code
import copy
def KNNImputer(df, n_neighbors=2): #TODO: n neighbors should be evaluated
    from sklearn import impute
    df_copy = copy.deepcopy(df)
    data = df[[f'ctr_{i}' for i in time]+[f'mg_{i}' for i in time]].values
    imputer = impute.KNNImputer(n_neighbors=n_neighbors)
    new_data = imputer.fit_transform(data)
    df_copy[[f'ctr_{i}' for i in time]+[f'mg_{i}' for i in time]] = new_data
    return df_copy
df_imput = KNNImputer(df) #TODO: this can be experimented
for gene in df_imput['Entry'].values:
    if df_imput.loc[df_imput['Entry']==gene].isna().values.any():
        print('still missing values: ', gene)